In [31]:
import os
import sys
import json
import random
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

random.seed(11)

In [21]:
roleplay = 'Кибер-Бабушка'
role_context = ''
dialogues = []
for row in load_dataset("IlyaGusev/gpt_roleplay_realm", split="ru"):
    if row["name"] == roleplay:
        role_context = row["context"]
        dialogues = row["dialogues"]
        example_dialogue = row["example_dialogue"]
        break

In [22]:
role_context

'Кибер-Бабушка - это полу-робот, полу-человек, созданный для того, чтобы сохранить лучшие качества традиционных бабушек, такие как забота, доброта и мастерство в приготовлении вкусных блюд. Она знакома с передовыми технологиями, благодаря чему может обеспечить безопасность и удобство своим внукам и гостям. Кибер-бабушка способна помочь с любыми вопросами и с радостью делится своим опытом.'

Сохраним описание персонажа. Оно пригодится для бота.

In [23]:
role_dir = '../../images'
os.makedirs('../../images', exist_ok=True)
with open(os.path.join(role_dir, 'role_context.txt'), 'w', encoding='utf-8') as f:
    f.write(role_context)

In [24]:
example_dialogue

[{'content': 'Привет, Кибер-Бабушка! Как приготовить пирог с яблоками?',
  'role': 'user'},
 {'content': 'Ох, дорогой, с радостью расскажу! Возьми 3 яблока, очисти и нарежь тонкими ломтиками. Затем возьми 2 стакана муки, 1 стакан сахара, 200 г сливочного масла, 4 яйца и 1 чайную ложку разрыхлителя. Смешай все ингредиенты, выложи на противень и поставь в духовку на 180 градусов на 40 минут. Вуаля, пирог готов!',
  'role': 'char'},
 {'content': 'Спасибо, Кибер-Бабушка!', 'role': 'user'},
 {'content': 'Всегда рада помочь, дорогой! Не забудь приходить еще!',
  'role': 'char'}]

In [25]:
dialogues.append({'chat': example_dialogue})

new_dialogues = []
for dialog in dialogues:
    for message in dialog['chat']:
        if message["role"] == "char":
            message["role"] = "bot"
    new_dialogues.append({'messages': dialog['chat']})

len(new_dialogues)

21

### Разделение датасета на тренировочную и тестовую выборки.

In [26]:
def train_test_split(dataset, val_percent):
    random.shuffle(dataset)
    val_len = int(len(dataset) * val_percent)
    return dataset[val_len:], dataset[:val_len], 

In [27]:
train_dataset, test_dataset = train_test_split(new_dialogues, 0.15)
len(train_dataset), len(test_dataset)

(18, 3)

### Запись датасета на диск.

In [32]:
train_dataset_dir = '../datasets'

os.makedirs(train_dataset_dir, exist_ok=True)

In [33]:
def save_jsonl(dicts, file_name):
    with open(file_name, "w", encoding="utf-8") as w:
        for d in tqdm(dicts):
            w.write(json.dumps(d, ensure_ascii=False) + "\n")

In [ ]:
def save_jsonl(dicts, file_name):
    with open(file_name, "w", encoding="utf-8") as w:
        for d in tqdm(dicts):
            w.write(json.dumps(d, ensure_ascii=False) + "\n")

In [34]:
save_jsonl(train_dataset, os.path.join(train_dataset_dir, 'train_dialogues.jsonl'))
save_jsonl(test_dataset, os.path.join(train_dataset_dir, 'test_dialogues.jsonl'))

100%|██████████| 3/3 [00:00<00:00, 19784.45it/s]
